In [ ]:

import numpy as np 
import pandas as pd 



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [ ]:
df=pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/train.csv',nrows=30000)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
sns.countplot(df['target'])

In [ ]:
df.corr()


In [ ]:
x=df.iloc[:,1:-1]
y=df['target']

In [ ]:
x.head()

## Outliers Removal

In [ ]:
set(df.dtypes)

In [ ]:
for col in x:

#         print(f"{col} underb value",x[col].quantile(0.10))
#         print(f"{col} over value",x[col].quantile(0.90))
        new_x=x.iloc[:]
        lower=x[col].quantile(0.10)
        upper=x[col].quantile(0.90)

        new_x[col]=np.where(new_x[col]<lower,lower,new_x[col])

        new_x[col]=np.where(new_x[col]>upper,upper,new_x[col])

In [ ]:
new_x.shape

In [ ]:

y.shape

In [ ]:
sns.countplot(y)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_scaled=scaler.fit_transform(new_x)

In [ ]:
x_scaled=pd.DataFrame(x_scaled)

In [ ]:
skf=StratifiedKFold(n_splits=5,random_state=0)

def result(clf):
    for train_index,test_index in skf.split(x_scaled,y):
        x_train,y_train=x_scaled.iloc[train_index],y.iloc[train_index]
        x_test,y_test=x_scaled.iloc[test_index],y.iloc[test_index]
        clf.fit(x_train,y_train)
        y_pred_test=clf.predict(x_test)
        y_pred_train=clf.predict(x_train)
        score_test=accuracy_score(y_pred_test,y_test)
        score_train=accuracy_score(y_pred_train,y_train)
        print("train score:",score_train)
        print("test score ",score_test)
    
            

In [ ]:
from sklearn.tree import DecisionTreeClassifier
result(DecisionTreeClassifier())


In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier()
params={
    'criterion':['gini','entropy'],
    'splitter':['best','random'], 
    'random_state':[0,1],
    'max_features':[10,270,40]
    
    
}
#max features he x mhadlya columns ppeksha jast nao asyla have

from sklearn.model_selection import GridSearchCV


grid=GridSearchCV(estimator=clf,param_grid=params,scoring='accuracy',cv=2)

grid.fit(x_scaled,y)

grid.best_estimator_

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
clf_dt=grid.best_estimator_
clf_rf=RandomForestClassifier()
clf_et=ExtraTreesClassifier(n_estimators=200,random_state=0)
clf_xb=XGBClassifier(n_estimators=100,max_depth=8)

In [ ]:
result(VotingClassifier(estimators=[('DT',clf_dt),
                                  ('RF',clf_rf),
                                  ('ET',clf_et),
                                  ('XGB',clf_xb)]))



# Testing Model on Test Data

In [ ]:
new_x_test=pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/test.csv')


In [ ]:
new_x_test

In [ ]:
new_x_test=new_x_test.iloc[:,1:-1]

In [ ]:
new_x_test['extra']=0

In [ ]:
new_x_test

In [ ]:
#Scaling the test file
for col in new_x_test:

#         print(f"{col} underb value",x[col].quantile(0.10))
#         print(f"{col} over value",x[col].quantile(0.90))
        new_x=new_x_test.iloc[:]
        lower=new_x_test[col].quantile(0.10)
        upper=new_x_test[col].quantile(0.90)
        new_x[col]=np.where(new_x[col]<lower,lower,new_x[col])

        new_x[col]=np.where(new_x[col]>upper,upper,new_x[col])
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_scaled_test=scaler.fit_transform(new_x)


In [ ]:
x_scaled_test.shape

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
clf_dt=DecisionTreeClassifier()
clf_rf=RandomForestClassifier()
clf_et=ExtraTreesClassifier(n_estimators=200,random_state=0)
clf_xb=XGBClassifier(n_estimators=100,max_depth=8)

In [ ]:
clf=VotingClassifier(estimators=[('DT',clf_dt),
                                  ('RF',clf_rf),
                                  ('ET',clf_et),
                                  ('XGB',clf_xb)])
clf.fit(x_scaled,y)
# actual_prediction=clf.predict(new_x_test)
# actual_prediction




In [ ]:
actual_prediction=clf.predict(x_scaled_test)

In [ ]:
actual_prediction=pd.DataFrame(actual_prediction)

In [ ]:
actual_prediction

In [ ]:
sam=pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv')

In [ ]:
sam['target']=actual_prediction.values

In [ ]:
sam.to_csv('submission2.csv',index=False)